# 데이터 엔지니어링 (Data Engineering)

## 1. 개요
이 노트북은 이커머스 배송 데이터(`Train.csv`)를 기반으로 머신러닝 모델 학습에 최적화된 데이터를 준비하는 과정을 담고 있습니다.
주요 작업은 다음과 같습니다:
- 데이터 로드 및 기본 확인
- 결측치 및 이상치 처리
- 피처 엔지니어링 (파생 변수 생성)
- 범주형 변수 인코딩
- 수치형 변수 스케일링
- 전처리 파이프라인 구축 및 저장

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Windows 기준)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

## 2. 데이터 로드 및 확인

In [2]:
# 데이터 로드
df = pd.read_csv('../data/Train.csv')

# 데이터 확인
display(df.head())
print(df.info())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB
None


## 3. 데이터 전처리 및 피처 엔지니어링

### 3.1. 결측치 및 중복값 확인

In [3]:
# 결측치 확인
print("Missing Values:\n", df.isnull().sum())

# 중복값 확인
print("\nDuplicate Rows:", df.duplicated().sum())

Missing Values:
 ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64

Duplicate Rows: 0


### 3.2. 파생 변수 생성 (Feature Creation)
EDA 결과를 바탕으로 다음 변수들을 생성합니다:
1. **Discount_Rate**: `Discount_offered` / `Cost_of_the_Product` (제품 가격 대비 할인율)
2. **Cost_per_Weight**: `Cost_of_the_Product` / `Weight_in_gms` (무게당 비용)
3. **Weight_Segment**: `Weight_in_gms`를 구간화 (Low, Medium, High)
4. **Call_Ratio**: `Customer_care_calls` / `Prior_purchases` (이전 구매 대비 문의 빈도)
5. **Value_Efficiency**: `Cost_of_the_Product` * `Product_importance_score` (중요도 가중치 부여한 가격 효율성)

In [4]:
def create_features(data):
    df_new = data.copy()
    
    # 1. Discount_Rate
    df_new['Discount_Rate'] = df_new['Discount_offered'] / df_new['Cost_of_the_Product']
    
    # 2. Cost_per_Weight
    df_new['Cost_per_Weight'] = df_new['Cost_of_the_Product'] / df_new['Weight_in_gms']
    
    # 3. Weight_Segment (EDA 인사이트 기반 : 4000g 기준 혹은 분포에 따라 나눔. 여기서는 3구간으로 임의 설정)
    # 참고: EDA에서 무게 분포가 bimodal 형태였음 (작은 무게, 큰 무게)
    # 0-2000: Low, 2000-4000: Medium, 4000+: High 로 가정하거나 qcut 사용
    # 여기서는 qcut을 사용하여 균등 분할
    df_new['Weight_Segment'] = pd.qcut(df_new['Weight_in_gms'], q=3, labels=['Low', 'Medium', 'High'])
    
    # 4. Call_Ratio
    df_new['Call_Ratio'] = df_new['Customer_care_calls'] / df_new['Prior_purchases']

    # 5. Value_Efficiency
    # Product_importance를 수치로 매핑하여 계산
    importance_map = {'low': 1, 'medium': 2, 'high': 3}
    df_new['Product_importance_score'] = df_new['Product_importance'].map(importance_map)
    df_new['Value_Efficiency'] = df_new['Cost_of_the_Product'] * df_new['Product_importance_score']
    # 임시 컬럼 제거
    df_new.drop(columns=['Product_importance_score'], inplace=True)
    
    # 타겟 변수 이름 변경 (편의상)
    if 'Reached.on.Time_Y.N' in df_new.columns:
        df_new.rename(columns={'Reached.on.Time_Y.N': 'Reached_on_Time'}, inplace=True)
        
    return df_new

df_fe = create_features(df)
df_fe.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached_on_Time,Discount_Rate,Cost_per_Weight,Weight_Segment,Call_Ratio,Value_Efficiency
0,1,D,Flight,4,2,177,3,low,F,44,1233,1,0.248588,0.143552,Low,1.333333,177
1,2,F,Flight,4,5,216,2,low,M,59,3088,1,0.273148,0.069948,Medium,2.000000,216
2,3,A,Flight,2,2,183,4,low,M,48,3374,1,0.262295,0.054238,Medium,0.500000,183
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1,0.056818,0.149533,Low,0.750000,352
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1,0.250000,0.074074,Medium,0.666667,368


### 3.3. 인코딩 및 스케일링 (Encoding & Scaling)

**범주형 변수 처리:**
- `Product_importance`: 순서가 있으므로 **Ordinal Encoding** (low=0, medium=1, high=2)
- `Gender`: **Label Encoding** (F=0, M=1)
- `Warehouse_block`, `Mode_of_Shipment`, `Weight_Segment`: **One-Hot Encoding**

**수치형 변수 처리:**
- Standard Scaling 적용

In [5]:
# ID 열 제거 (모델링에 불필요)
if 'ID' in df_fe.columns:
    df_fe = df_fe.drop(columns=['ID'])

# 변수 구분
target_col = 'Reached_on_Time'
ordinal_cols = ['Product_importance']
categorical_cols = ['Warehouse_block', 'Mode_of_Shipment', 'Gender', 'Weight_Segment']
numeric_cols = [
    'Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 
    'Discount_offered', 'Weight_in_gms', 'Discount_Rate', 'Cost_per_Weight', 'Call_Ratio',
    'Value_Efficiency'
]

# Pipeline 구축을 위한 전처리 클래스 정의
from sklearn.base import BaseEstimator, TransformerMixin

class CustomFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        # 파생 변수 생성 로직을 여기에 포함할 수도 있으나, 
        # 위에서 이미 생성했으므로 여기서는 패스하거나 필요시 추가
        # 실제 파이프라인에서는 create_features 로직이 여기 들어와야 함
        return data

# 인코더 및 스케일러 정의
# Product_importance (low, medium, high) 순서 매핑
ordinal_encoder = OrdinalEncoder(categories=[['low', 'medium', 'high']])

preprocessor = ColumnTransformer(
    transformers=[
        ('ord', ordinal_encoder, ordinal_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),
        ('num', StandardScaler(), numeric_cols)
    ],
    remainder='passthrough' # 타겟 변수는 그대로 둠 (X, y 분리 시 처리)
)

# 전체 파이프라인 (만약 Feature Creation도 파이프라인에 넣고 싶다면 FunctionTransformer 사용 가능)
# 여기서는 df_fe를 이미 생성했으므로 전처리만 수행

# X, y 분리
X = df_fe.drop(columns=[target_col])
y = df_fe[target_col]

# 전처리 수행
X_processed = preprocessor.fit_transform(X)

# 컬럼명 복원 (OneHotEncoder 등으로 인해 컬럼명 변경됨)
# Ordinal
ord_feature_names = ordinal_cols
# OneHot
cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
# Numeric
num_feature_names = numeric_cols

final_cols = list(ord_feature_names) + list(cat_feature_names) + list(num_feature_names)

X_processed_df = pd.DataFrame(X_processed, columns=final_cols)
X_processed_df.head()

,Product_importance,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Gender_M,Weight_Segment_Low,Weight_Segment_Medium,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Discount_Rate,Cost_per_Weight,Call_Ratio,Value_Efficiency
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.047711,-0.700755,-0.690722,-0.372735,1.889983,-1.468240,1.924300,1.150475,0.106910,-1.032936
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,-0.047711,1.421578,0.120746,-1.029424,2.815636,-0.333893,2.189294,-0.165997,1.436478,-0.779318
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.799887,-0.700755,-0.565881,0.283954,2.136824,-0.159002,2.072196,-0.446982,-1.555050,-0.993918
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.923799,0.006689,-0.711529,0.283954,-0.208162,-1.502484,-0.144777,1.257440,-1.056462,0.105092
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.799887,-0.700755,-0.545074,-0.372735,2.013404,-0.703244,1.939539,-0.092202,-1.222658,0.209140


## 4. 데이터 저장

전처리가 완료된 데이터를 저장하여 모델링 파트에 전달합니다.

In [6]:
# 타겟 변수 다시 결합
final_df = pd.concat([X_processed_df, y], axis=1)

display(final_df.head())

# 저장 경로 설정 (data 폴더)
import os
save_dir = './data'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_path = os.path.join(save_dir, 'processed_train.csv')
final_df.to_csv(save_path, index=False)

print(f"Processed data saved to {save_path}")

,Product_importance,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Gender_M,Weight_Segment_Low,Weight_Segment_Medium,...,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Discount_Rate,Cost_per_Weight,Call_Ratio,Value_Efficiency,Reached_on_Time
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.700755,-0.690722,-0.372735,1.889983,-1.468240,1.924300,1.150475,0.106910,-1.032936,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.421578,0.120746,-1.029424,2.815636,-0.333893,2.189294,-0.165997,1.436478,-0.779318,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,-0.700755,-0.565881,0.283954,2.136824,-0.159002,2.072196,-0.446982,-1.555050,-0.993918,1
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.006689,-0.711529,0.283954,-0.208162,-1.502484,-0.144777,1.257440,-1.056462,0.105092,1
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.700755,-0.545074,-0.372735,2.013404,-0.703244,1.939539,-0.092202,-1.222658,0.209140,1


Processed data saved to ./data\processed_train.csv
